# Cabeçalho

## Resumo do trabalho

## Importando bibliotecas

In [1]:
import pandas as pd

## Importando o dado

In [2]:
ls data/

Fat_Supply_Quantity_Data.csv      Protein_Supply_Quantity_Data.csv
Food_Supply_kcal_Data.csv         readme.txt
Food_Supply_Quantity_kg_Data.csv  Supply_Food_Data_Descriptions.csv


In [3]:
fat_df = pd.read_csv('./data/Fat_Supply_Quantity_Data.csv')

In [4]:
fat_df.head()

,Country,Alcoholic Beverages,Animal Products,Animal fats,"Aquatic Products, Other",Cereals - Excluding Beer,Eggs,"Fish, Seafood",Fruits - Excluding Wine,Meat,...,Vegetable Oils,Vegetables,Obesity,Undernourished,Confirmed,Deaths,Recovered,Active,Population,Unit (all except Population)
0,Afghanistan,0.0,21.6397,6.2224,0.0,8.0353,0.6859,0.0327,0.4246,6.1244,...,17.0831,0.3593,4.5,29.8,0.021411,0.000492,0.002445,0.018474,38042000.0,%
1,Albania,0.0,32.0002,3.4172,0.0,2.6734,1.6448,0.1445,0.6418,8.7428,...,9.2443,0.6503,22.3,6.2,0.033730,0.001085,0.026522,0.006123,2858000.0,%
2,Algeria,0.0,14.4175,0.8972,0.0,4.2035,1.2171,0.2008,0.5772,3.8961,...,27.3606,0.5145,26.6,3.9,0.017375,0.001309,0.009142,0.006925,43406000.0,%
3,Angola,0.0,15.3041,1.3130,0.0,6.5545,0.1539,1.4155,0.3488,11.0268,...,22.4638,0.1231,6.8,25,0.000165,0.000010,0.000054,0.000102,31427000.0,%
4,Antigua and Barbuda,0.0,27.7033,4.6686,0.0,3.2153,0.3872,1.5263,1.2177,14.3202,...,14.4436,0.2469,19.1,NaN,0.025773,0.003093,0.019588,0.003093,97000.0,%


In [5]:
protein_df = pd.read_csv('./data/Protein_Supply_Quantity_Data.csv')

In [6]:
protein_df.head()

,Country,Alcoholic Beverages,Animal Products,Animal fats,"Aquatic Products, Other",Cereals - Excluding Beer,Eggs,"Fish, Seafood",Fruits - Excluding Wine,Meat,...,Vegetables,Miscellaneous,Obesity,Undernourished,Confirmed,Deaths,Recovered,Active,Population,Unit (all except Population)
0,Afghanistan,0.0000,9.7523,0.0277,0.0,35.9771,0.4067,0.0647,0.5824,3.1337,...,1.1370,0.0462,4.5,29.8,0.021411,0.000492,0.002445,0.018474,38042000.0,%
1,Albania,0.1840,27.7469,0.0711,0.0,14.2331,1.8069,0.6274,1.2757,7.6582,...,3.2456,0.0544,22.3,6.2,0.033730,0.001085,0.026522,0.006123,2858000.0,%
2,Algeria,0.0323,13.8360,0.0054,0.0,26.5633,1.2916,0.6350,1.1624,3.5088,...,3.1267,0.1399,26.6,3.9,0.017375,0.001309,0.009142,0.006925,43406000.0,%
3,Angola,0.6285,15.2311,0.0277,0.0,20.3882,0.1756,5.4436,1.2754,7.6248,...,0.8133,0.0924,6.8,25,0.000165,0.000010,0.000054,0.000102,31427000.0,%
4,Antigua and Barbuda,0.1535,33.1901,0.1289,0.0,10.5108,0.4850,8.2146,1.2586,16.0670,...,1.6024,0.2947,19.1,NaN,0.025773,0.003093,0.019588,0.003093,97000.0,%


In [7]:
kcal_df = pd.read_csv('./data/Food_Supply_kcal_Data.csv')

In [8]:
kcal_df.head()

,Country,Alcoholic Beverages,Animal Products,Animal fats,"Aquatic Products, Other",Cereals - Excluding Beer,Eggs,"Fish, Seafood",Fruits - Excluding Wine,Meat,...,Vegetable Oils,Vegetables,Obesity,Undernourished,Confirmed,Deaths,Recovered,Active,Population,Unit (all except Population)
0,Afghanistan,0.0000,4.7774,0.8504,0.0,37.1186,0.1501,0.0000,1.4757,1.2006,...,2.3012,0.7504,4.5,29.8,0.021411,0.000492,0.002445,0.018474,38042000.0,%
1,Albania,0.9120,16.0930,1.0591,0.0,16.2107,0.8091,0.1471,3.8982,3.8688,...,2.8244,2.7508,22.3,6.2,0.033730,0.001085,0.026522,0.006123,2858000.0,%
2,Algeria,0.0896,6.0326,0.1941,0.0,25.0112,0.4181,0.1195,3.1805,1.2543,...,5.7638,2.0457,26.6,3.9,0.017375,0.001309,0.009142,0.006925,43406000.0,%
3,Angola,1.9388,4.6927,0.2644,0.0,18.3521,0.0441,0.8372,2.3133,2.9302,...,4.2741,0.3525,6.8,25,0.000165,0.000010,0.000054,0.000102,31427000.0,%
4,Antigua and Barbuda,2.3041,15.3672,1.5429,0.0,13.7215,0.2057,1.7280,3.6824,7.0356,...,4.6904,1.2960,19.1,NaN,0.025773,0.003093,0.019588,0.003093,97000.0,%


In [9]:
quantity_kg_df = pd.read_csv('./data/Food_Supply_Quantity_kg_Data.csv')

In [10]:
quantity_kg_df.head()

,Country,Alcoholic Beverages,Animal fats,Animal Products,"Aquatic Products, Other",Cereals - Excluding Beer,Eggs,"Fish, Seafood",Fruits - Excluding Wine,Meat,...,Vegetables,Vegetal Products,Obesity,Undernourished,Confirmed,Deaths,Recovered,Active,Population,Unit (all except Population)
0,Afghanistan,0.0014,0.1973,9.4341,0.0,24.8097,0.2099,0.0350,5.3495,1.2020,...,6.7642,40.5645,4.5,29.8,0.021411,0.000492,0.002445,0.018474,38042000.0,%
1,Albania,1.6719,0.1357,18.7684,0.0,5.7817,0.5815,0.2126,6.7861,1.8845,...,11.7753,31.2304,22.3,6.2,0.033730,0.001085,0.026522,0.006123,2858000.0,%
2,Algeria,0.2711,0.0282,9.6334,0.0,13.6816,0.5277,0.2416,6.3801,1.1305,...,11.6484,40.3651,26.6,3.9,0.017375,0.001309,0.009142,0.006925,43406000.0,%
3,Angola,5.8087,0.0560,4.9278,0.0,9.1085,0.0587,1.7707,6.0005,2.0571,...,2.3041,45.0722,6.8,25,0.000165,0.000010,0.000054,0.000102,31427000.0,%
4,Antigua and Barbuda,3.5764,0.0087,16.6613,0.0,5.9960,0.2274,4.1489,10.7451,5.6888,...,5.4495,33.3233,19.1,NaN,0.025773,0.003093,0.019588,0.003093,97000.0,%


In [11]:
description_df = pd.read_csv('./data/Supply_Food_Data_Descriptions.csv')

In [12]:
description_df.head()

,Categories,Items
0,Alcoholic Beverages,"Alcohol, Non-Food; Beer; Beverages, Alcoholic;..."
1,Animal fats,"Butter, Ghee; Cream; Fats, Animals, Raw; Fish,..."
2,Animal Products,"Aquatic Animals, Others; Aquatic Plants; Bovin..."
3,"Aquatic Products, Other","Aquatic Animals, Others; Aquatic Plants; Meat,..."
4,Cereals - Excluding Beer,"Barley and products; Cereals, Other; Maize and..."


## Juntando datasets

In [13]:
dfs = [fat_df, protein_df, kcal_df, quantity_kg_df, description_df]

In [14]:
for df in dfs:
    print (df.shape)

(170, 32)
(170, 32)
(170, 32)
(170, 32)
(23, 2)


In [15]:
cols = {}
i = 0
for df in dfs[:-1]:
    i += 1
    cols['df'+str(i)] = list(df.columns)
df_cols = pd.DataFrame(cols)

In [16]:
df_cols

,df1,df2,df3,df4
0,Country,Country,Country,Country
1,Alcoholic Beverages,Alcoholic Beverages,Alcoholic Beverages,Alcoholic Beverages
2,Animal Products,Animal Products,Animal Products,Animal fats
3,Animal fats,Animal fats,Animal fats,Animal Products
4,"Aquatic Products, Other","Aquatic Products, Other","Aquatic Products, Other","Aquatic Products, Other"
5,Cereals - Excluding Beer,Cereals - Excluding Beer,Cereals - Excluding Beer,Cereals - Excluding Beer
6,Eggs,Eggs,Eggs,Eggs
7,"Fish, Seafood","Fish, Seafood","Fish, Seafood","Fish, Seafood"
8,Fruits - Excluding Wine,Fruits - Excluding Wine,Fruits - Excluding Wine,Fruits - Excluding Wine
9,Meat,Meat,Meat,Meat


In [17]:
shared_cols = ['Country', 'Obesity', 'Undernourished', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Population', 'Unit (all except Population)']

In [18]:
dataset = fat_df[shared_cols].copy()

In [19]:
dataset.head()

,Country,Obesity,Undernourished,Confirmed,Deaths,Recovered,Active,Population,Unit (all except Population)
0,Afghanistan,4.5,29.8,0.021411,0.000492,0.002445,0.018474,38042000.0,%
1,Albania,22.3,6.2,0.033730,0.001085,0.026522,0.006123,2858000.0,%
2,Algeria,26.6,3.9,0.017375,0.001309,0.009142,0.006925,43406000.0,%
3,Angola,6.8,25,0.000165,0.000010,0.000054,0.000102,31427000.0,%
4,Antigua and Barbuda,19.1,NaN,0.025773,0.003093,0.019588,0.003093,97000.0,%


In [20]:
cols = list(fat_df.columns)

In [21]:
cols = ['Country'] + [i for i in cols if i not in shared_cols]

In [22]:
fat_df = fat_df[cols].copy()

In [23]:
protein_df = protein_df[cols].copy()

In [24]:
kcal_df = kcal_df[cols].copy()

In [25]:
quantity_kg_df = quantity_kg_df[cols].copy()

In [26]:
fat_cols_map = {i:i+'_fat' for i in cols if i != 'Country'}

In [27]:
protein_cols_map = {i:i+'_protein' for i in cols if i != 'Country'}

In [28]:
kcal_cols_map = {i:i+'_kcal' for i in cols if i != 'Country'}

In [29]:
quant_cols_map = {i:i+'_quantity' for i in cols if i != 'Country'}

In [30]:
fat_df.rename(fat_cols_map, axis=1, inplace=True)

In [31]:
protein_df.rename(protein_cols_map, axis=1, inplace=True)

In [32]:
kcal_df.rename(kcal_cols_map, axis=1, inplace=True)

In [33]:
quantity_kg_df.rename(quant_cols_map, axis=1, inplace=True)

In [34]:
dataset = dataset.merge(fat_df, how='left', on='Country', copy=False)

In [35]:
dataset = dataset.merge(protein_df, how='left', on='Country', copy=False)

In [36]:
dataset = dataset.merge(kcal_df, how='left', on='Country', copy=False)

In [37]:
dataset = dataset.merge(quantity_kg_df, how='left', on='Country', copy=False)

In [38]:
dataset.shape

(170, 101)

## Analisando o dado

In [39]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170 entries, 0 to 169
Columns: 101 entries, Country to Vegetables_quantity
dtypes: float64(98), object(3)
memory usage: 135.5+ KB


In [40]:
dataset.dtypes.value_counts()

float64    98
object      3
dtype: int64

In [41]:
data_num = dataset.select_dtypes(include = ['float64', 'int64'])
data_cat = dataset.select_dtypes(include = ['object'])

In [42]:
data_cat.head()

,Country,Undernourished,Unit (all except Population)
0,Afghanistan,29.8,%
1,Albania,6.2,%
2,Algeria,3.9,%
3,Angola,25,%
4,Antigua and Barbuda,NaN,%


In [43]:
data_num.describe()

,Obesity,Confirmed,Deaths,Recovered,Active,Population,Alcoholic Beverages_fat,Animal Products_fat,Animal fats_fat,"Aquatic Products, Other_fat",...,Pulses_quantity,Spices_quantity,Starchy Roots_quantity,Stimulants_quantity,Sugar Crops_quantity,Sugar & Sweeteners_quantity,Treenuts_quantity,Vegetal Products_quantity,Vegetable Oils_quantity,Vegetables_quantity
count,167.000000,161.000000,161.000000,161.000000,161.000000,1.700000e+02,170.000000,170.000000,170.000000,170.000000,...,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000
mean,18.707784,0.083872,0.004454,0.042839,0.036576,4.408395e+07,0.001035,20.695714,4.138451,0.000450,...,0.537131,0.091011,5.335371,0.201946,0.095846,2.799115,0.117474,37.814834,0.851554,6.085912
std,9.633557,0.127583,0.011492,0.085173,0.067335,1.556265e+08,0.009569,8.002713,3.287462,0.004039,...,0.601111,0.121898,5.571355,0.189239,0.379936,1.545951,0.146143,5.852943,0.445335,3.560148
min,2.100000,0.000047,0.000000,0.000000,0.000000,5.300000e+04,0.000000,5.018200,0.026200,0.000000,...,0.000000,0.000000,0.679600,0.004200,0.000000,0.366600,0.000000,23.113200,0.091500,0.857000
25%,8.500000,0.006013,0.000147,0.002235,0.001395,2.822750e+06,0.000000,14.885800,1.674225,0.000000,...,0.129650,0.016500,2.023825,0.083325,0.000000,1.721025,0.020750,33.552775,0.507700,3.616500
50%,21.200000,0.022222,0.000545,0.009806,0.008685,1.014600e+07,0.000000,20.943050,3.312350,0.000000,...,0.300800,0.042550,3.108650,0.159450,0.000000,2.601650,0.079950,37.900700,0.774450,5.028100
75%,25.700000,0.116800,0.002828,0.048328,0.030238,3.169250e+07,0.000000,26.866950,6.227575,0.000000,...,0.734900,0.123150,5.745875,0.261800,0.000000,3.782150,0.148650,42.762775,1.070900,7.801650
max,45.600000,0.640484,0.079857,0.601290,0.353151,1.398030e+09,0.097600,36.901800,14.937300,0.051100,...,3.483800,0.662600,27.712800,1.282300,3.067700,9.725900,0.756900,48.258500,2.202600,19.299500


In [44]:
dataset.isnull().sum()

Country                        0
Obesity                        3
Undernourished                 7
Confirmed                      9
Deaths                         9
                              ..
Sugar & Sweeteners_quantity    0
Treenuts_quantity              0
Vegetal Products_quantity      0
Vegetable Oils_quantity        0
Vegetables_quantity            0
Length: 101, dtype: int64

In [45]:
for col, val in zip(dataset.isnull().sum().index.to_list(), list(dataset.isnull().sum().values)):
    if val > 0:
        print (col, val) 

Obesity 3
Undernourished 7
Confirmed 9
Deaths 9
Recovered 9
Active 9


In [46]:
dataset = dataset[dataset['Confirmed'].isnull() != True]

In [47]:
dataset.shape

(161, 101)

In [48]:
for col, val in zip(dataset.isnull().sum().index.to_list(), list(dataset.isnull().sum().values)):
    if val > 0:
        print (col, val) 

Obesity 1
Undernourished 7


In [49]:
dataset['Undernourished'].unique()

array(['29.8', '6.2', '3.9', '25', nan, '4.6', '4.3', '<2.5', '14.7',
       '7.5', '10.1', '17.1', '26.4', '3.6', '20', '12.6', '16.4', '9.9',
       '59.6', '37.5', '2.7', '8.5', '4.8', '40.3', '19', '5.6', '18.9',
       '9.5', '7.9', '4.5', '9', '2.9', '20.6', '3.7', '10.5', '10.2',
       '5.5', '15.2', '16.5', '28', '8.1', '49.3', '12.9', '14.5', '8.3',
       '4.9', '29', '8', '12.2', '29.4', '2.8', '7.1', '11', '13.1',
       '37.2', '44.4', '17.5', '2.5', '10.3', '6.3', '10.4', '6.5',
       '13.4', '3.4', '27.9', '27.3', '8.7', '17', '3.2', '6.8', '20.3',
       '10', '10.7', '9.7', '13.3', '36.8', '5.7', '7', '11.3', '25.6',
       '20.1', '3.5', '7.8', '24.9', '16.1', '41', '2.6', '30.7', '21.2',
       '9.3', '38.9', '46.7', '51.3'], dtype=object)

In [50]:
dataset.loc[dataset['Undernourished'] == '<2.5', 'Undernourished'] = '2.5'

In [51]:
dataset['Undernourished'] = dataset['Undernourished'].astype(float)

In [52]:
dataset['Obesity'].unique()

array([ 4.5, 22.3, 26.6,  6.8, 19.1, 28.5, 20.9, 30.4, 21.9, 19.9, 32.1,
        3.4, 24.8, 24.5, 22.4,  8.2, 18.7, 19.4, 16.1, 27.4, 10.6,  3.5,
        9.5, 31.3,  6.3,  4.8, 28.8,  6.6, 22.1,  8.4, 25.7,  9. , 27.1,
       26.7, 22.6, 21.3, 12.2, 28.2, 26.9, 19.3, 31.1, 22.7, 23.8, 13.5,
        3.6, 30. , 24.9, 23.2, 13.4,  8.7, 23.3,  9.7, 20.2, 18.8, 19.2,
       20.5, 28.6, 23.1,  3.8,  6.9, 25.5, 22.9, 24.4,  4.4, 33.4,  6. ,
        4.9, 37. , 15.4,  8.6, 28.4, 24.2,  4.7, 15.3,  7.9,  7.1, 31. ,
       11.3, 11.5, 19.6, 25.6, 15. , 32. , 21.8,  7.8, 23.9, 25. , 22.5,
       19. , 20.1, 19.8, 35. ,  7.4, 23.5,  7.5, 27. ,  5.4, 26.5, 21.2,
        nan, 12.6, 10.8,  2.9, 19.7, 27.3, 32.2,  4.1, 26.1, 29.9, 29.5,
       37.3, 28.9, 25.2,  2.1, 14.1,  6.5, 12.3])

In [53]:
dataset['Obesity'] = dataset['Obesity'].astype(float)

In [54]:
dataset['Obesity'].fillna(dataset['Obesity'].mean(), inplace=True)

In [55]:
dataset['Undernourished'].fillna(dataset['Undernourished'].mean(), inplace=True)

In [56]:
dataset.isnull().sum().unique()

array([0])

## Golden features

In [57]:
targets = ['Confirmed', 'Deaths', 'Recovered', 'Active']

In [58]:
for t in targets:
    data_num = dataset.select_dtypes(include = ['float64', 'int64'])
    data_corr = data_num.corr()[t]
    golden_features_list = data_corr[abs(data_corr) > 0.2].sort_values(ascending=False)
    print (t, golden_features_list)

Confirmed Confirmed                            1.000000
Recovered                            0.823831
Active                               0.730086
Deaths                               0.717421
Animal Products_kcal                 0.482963
Animal Products_protein              0.471756
Animal Products_quantity             0.454655
Eggs_kcal                            0.429222
Milk - Excluding Butter_protein      0.423439
Obesity                              0.419346
Animal fats_quantity                 0.405072
Milk - Excluding Butter_kcal         0.401307
Animal fats_kcal                     0.395164
Stimulants_kcal                      0.386813
Eggs_protein                         0.384428
Milk - Excluding Butter_quantity     0.379119
Alcoholic Beverages_kcal             0.369625
Animal fats_fat                      0.350306
Animal Products_fat                  0.344300
Meat_kcal                            0.341846
Treenuts_kcal                        0.336915
Meat_protein            

## Remover outliers